# Configurando e testando prompts

In [1]:
#@title Instalar dependências e carregar pacotes do git
# carregando o get_git para importar utilitários do github
import os
if not os.path.isfile('./get_git.py'):
   !curl https://raw.githubusercontent.com/luizanisio/llms/refs/heads/main/util/get_git.py -o ./get_git.py
   import get_git
   get_git.sync()
else:
   import get_git
# Verifica e atualiza dependências (como o Unsloth, se solicitado)
get_git.deps(unsloth=False) # instala unsloth se True, Transformers, Rouge e Levenshtein se precisar
from src.util import Util
Util.dados_python()



 ✅ Levenshtein e Rouge instalados _o/
python-Levenshtein rouge-score OK _o/
[OK] Dependências de análise instaladas _o/
Verificando dependências ....
imports transformers ok ___o/
||||||||||||||||||||||
||| ATENÇÃO  
||| * Unsloth não verificado, utilize o parâmetro unsloth=True para instalar dependências do Unsloth!
||| ** Modelos como o Gemma 3 funcionam melhor com GPU L4 em diante ou com Unsloth
||| *** Considere instalar Unsloth para treinamento 
||||||||||||||||||||||


{'python': {'version': '3.11.13',
  'env': 'nenhum',
  'hardware': {'cpus_fisicas': 6,
   'cpus_logicas': 12,
   'mem_disponivel_gb': 80.68,
   'mem_total_gb': 83.476,
   'disco_uso_%': 16.3,
   'cpu_uso_%': 14.9,
   'cpu_uso_processo_%': 0.0}}}

In [3]:
#@title Lista os modelos disponíveis
from src.util_prompt import Prompt, UtilLLM
Prompt.listar_modelos()
print('-='*15)
UtilLLM.verifica_versao()

Modelos disponíveis:

- MODELO_GEMMA3_1B: google/gemma-3-1b-it
- MODELO_GEMMA3_4B: google/gemma-3-4b-it
- MODELO_GEMMA3_12B: google/gemma-3-12b-it
- MODELO_GEMMA3_27B: google/gemma-3-27b-it
- MODELO_QWEN_7B: Qwen/Qwen2.5-7B-Instruct-1M
- MODELO_JUREMA_7B: Jurema-br/Jurema-7B
- MODELO_QWEN_14B: Qwen/Qwen2.5-14B-Instruct-1M
- MODELO_DEEPSEEK_1_5B: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
- MODELO_DEEPSEEK_70B: deepseek-ai/DeepSeek-R1-Distill-Llama-70B
- MODELO_DEEPSEEK_32B: deepseek-ai/DeepSeek-R1-Distill-Qwen-32B
- MODELO_DEEPSEEK_14B: deepseek-ai/DeepSeek-R1-Distill-Qwen-14B
- MODELO_DEEPSEEK_7B: deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
- MODELO_DEEPSEEK_8B: deepseek-ai/DeepSeek-R1-Distill-Llama-8B
- MODELO_OSS_20B: openai/gpt-oss-20b
- MODELO_OSS_120B: openai/gpt-oss-120b
- MODELO_LLAMA_4_SCOUT_16B: meta-llama/Llama-4-Scout-17B-16E-Instruct
- MODELO_LLAMA_3_3_70B: meta-llama/Llama-3.3-70B-Instruct
- MODELO_LLAMA_3_2_1B: meta-llama/Llama-3.2-1B-Instruct
- MODELO_LLAMA_3_2_3B: meta-llam

In [ ]:
#@title Escolhe e carrega o modelo
from src.util_prompt import Prompt
# Use os modelos indicados em Prompt.listar_modelos()
# Exemplos:
# Prompt.modelos.MODELO_DEEPSEEK_1_5B
# Prompt.modelos.MODELO_GEMMA3_1B
p = Prompt(modelo=Prompt.modelos.MODELO_GEMMA3_1B,
           max_seq_length=2048,
           usar_unsloth=False)


In [7]:
#@title Pergunta ao modelo
res = p.prompt('Complete os próximos dois números da sequência e explique de forma objetiva em uma frase: 1, 1, 2, 3, 5, 8 ...')
print('RESPOSTA:', res)


`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True, 'top_p': 0.95}. If this is not desired, please set these values explicitly.
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


RESPOSTA: O próximo número é 13, pois a sequência é a sequência de Fibonacci, onde cada número é a soma dos dois números anteriores.


In [8]:
#@title Resposta em json do modelo
import json
pergunta = '''Retorne um json válido com a estrutrua {"mensagem": com a mensagem do usuário, "itens": com uma lista de itens quando ele enumerar algo }
              Mensagem do usuário: Eu preciso comprar abacaxi, pera e 2L de leite.'''
r = p.prompt_to_json(pergunta)
print(json.dumps(r, indent=2, ensure_ascii=False))

{
  "mensagem": "Eu preciso comprar abacaxi, pera e 2L de leite.",
  "itens": [
    "abacaxi",
    "pera",
    "2L de leite"
  ],
  "usage": {
    "input_tokens": 64,
    "output_tokens": 50,
    "time": 48.43174362182617
  }
}


# Configurando agentes

In [5]:
#@title Tarefa básica de exemplo
from IPython.display import clear_output
from src.util_agentes import ResolverTarefas, Tarefa, AgentesToolsBasicos
from src.util_prompt import Prompt
print('Carregando modelo ...')
p = Prompt(modelo=Prompt.modelos.MODELO_DEEPSEEK_7B, max_seq_length=4096)
clear_output()
print('|' * 30)
print(f"O modelo está no dispositivo: {next(p._model.parameters()).device}")
print('|' * 30)

print(f'Modelo {p.modelo} carregado _o/')
print('Preparando agentes ...')
servicos = AgentesToolsBasicos.get_servicos_basicos()
print('Definindo tarefa ...')

tarefa = Tarefa(nome='Tarefa 001',
                descricao='Descobrir que dia é hoje e encontrar informações sobre o que é Xibunfa e como podemos usar Xibunfa para limpar o chão.',
                objetivo='Descrever o que é Xibunfa e explicar como Xibunfa pode ser usada na limpeza do chão.',
                servicos_disponiveis=servicos,
                call_llm = p.prompt_to_json)

rt = ResolverTarefas([tarefa])
print('Resolvendo tarefa ...\n---------------------')
rt.resolver()

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


||||||||||||||||||||||||||||||
O modelo está no dispositivo: cuda:0
||||||||||||||||||||||||||||||
Modelo deepseek-ai/DeepSeek-R1-Distill-Qwen-7B carregado _o/
Preparando agentes ...
Definindo tarefa ...
Resolvendo tarefa ...
---------------------
23:24:10|RESOLVENDO TAREFA - : Tarefa 001 com 2 serviços disponíveis
23:24:10|DESCOBERTA - : Descobrindo serviços para a tarefa Tarefa 001


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


23:24:20|DESCOBERTA CONCLUÍDA - : Serviços descobertos para a tarefa Tarefa 001: 2
23:24:20|SERVIÇO SUGERIDO - : Busca - O usuário deseja descobrir o que é Xibunfa e como pode ser usado para limpar o chão.
23:24:20|SERVIÇO SUGERIDO - : DataHora - Para compreender melhor como Xibunfa pode ser employed, é útil obter a data atual.
23:24:20|ACIONANDO SERVIÇOS - : Busca,DataHora
23:24:20|PREPARANDO SERVIÇO - : Busca: {"palavras": ["Xibunfa", "limpar chão"]}
23:24:20|SERVIÇO ACIONADO - : Busca: {"palavras": ["Xibunfa", "limpar chão"]}
23:24:20|PREPARANDO SERVIÇO - : DataHora: {}
23:24:20|SERVIÇO ACIONADO - : DataHora: {}
23:24:20|CONHECIMENTO - : Calendário com dia, mês e ano | pg. None | score 0
ACIONANDO AGENTE DE TAREFA: Padrão


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


AVALIANDO RESPOSTA LLM:
{
  "solucao_tarefa": "Hoje é o dia 17 de Agosto de 2025. Xibunfa é uma substância que pode ser usada em tratamento de danos como areia ou madeira, mas não em limpeza do chão em geral.",
  "contribuicao": "Forneceu a data atual e expô a natureza de Xibunfa e suas limitações na limpeza do chão.",
  "concluida": true,
  "usage": {
    "input_tokens": 614,
    "output_tokens": 344,
    "time": 10.85403060913086,
    "think": "Primeiro, preciso identificar o dia atual usando a informação da página 1, página 1, url: https://example.com/calendario. O dia atual é 17 de Agosto de 2025.\n\nEm seguida, consulto a página 2, página 2, url: https://example.com/xibunfa, para obter informações sobre Xibunfa. Descobro que Xibunfa é uma substância que pode ser usada na limpeza do chão, mas também pode causar danos se invadido.\n\nPara resolver o objetivo, preciso expor como Xibunfa pode ser employed para limpar o chão. De acordo com a página 2, Xibunfa é employed em tratamento d

In [2]:
#@title Imprimindo solução da tarefa
tarefa.print(True)

TAREFA: Tarefa 001
DESCRIÇÃO: Descobrir que dia é hoje e encontrar informações sobre o que é Xibunfa e como podemos usar Xibunfa para limpar o chão.
OBJETIVO: Descrever o que é Xibunfa e explicar como Xibunfa pode ser usada na limpeza do chão.
---------------------------------------
CONHECIMENTO UTILIZADO:
	 - Calendário com dia, mês e ano (pg: None | sc: 0): Hoje é o dia 17 de August de 2025
---------------------------------------
SOLUÇÃO COMPLETA: Xibunfa é um produto químico utilizado na limpeza de pisos. É eficaz na remoção de sujeiras difíceis e manchas persistentes. Para utilizá-lo, basta misturá-lo com água conforme as instruções do fabricante e aplicá-lo no piso. Deixe agir por alguns minutos antes de escovar e enxaguar.
